# 读取ROOT的tree数据

In [1]:
%jsroot on
//声明tree中的所存的数据
Double_t tu, td;
Double_t qu, qd;
Double_t ctof,x;
int pid;
TFile *ipf = new TFile("tree.root");//打开ROOT文件
TTree *tree = (TTree*)ipf->Get("tree");//得到tree的指针

tree->SetBranchAddress("tu", &tu);   
tree->SetBranchAddress("td", &td);
tree->SetBranchAddress("qu", &qu);
tree->SetBranchAddress("qd", &qd);
tree->SetBranchAddress("ctof", &ctof);
tree->SetBranchAddress("x", &x);
tree->SetBranchAddress("pid", &pid);
Long64_t nentries=tree->GetEntries();

# 1.位置测量

   ###       1.1 利用时间信息确定位置tx

探测器两端时间信号：
- $t_u=TOF+(L-x)/v_{sc}+t_{u-off}$
- $t_d=TOF+(L+x)/v_{sc}+t_{u-off}$
- $x=(t_d-t_u+（t_{u-off}-t_{u-off}）)*v_{sc}/2$
- $x=[-100,100]$

#    $$x=(t_d-t_u + C_1)*C_{2}$$
#    $$x=a(t_d-t_u)+b$$

结论：

   1.位置与探测器两极电子学原件接收到信号的时间差$(t_d-t_u）$成正比

   2.时间差 $(t_d-t_u+C_1)$ 的中心在0处
   
   3.$(t_d-t_u + C_1)*C_{2} = [-100,100]$
   
   4.$a = C_2,$ $b =C_1C_2 $

In [2]:
TCanvas *c = new TCanvas();
tree->Draw("td-tu:x", "", "colz");
c->Draw();

结论：上图印证了位置与探测器两极电子学原件接收到信号的时间差$(t_d-t_u）$成正比的结论

In [3]:
tree->Draw("td-tu>>hdt(500, -20, 50)");//绘制td-tu的直方图
c->Draw();

# 微分法确定时间差$(t_d-t_u）$的边界

In [4]:
TH1D *dtd = new TH1D("dtd", "dt/dx", 141, -20.25, 50.25);

In [5]:
for(int i=1; i<hdt->GetNbinsX(); i++) 
{   //h->GetNbinsX()  获取 bin 总数   bin数从1开始
    Double_t df = hdt->GetBinContent(i+1) - hdt->GetBinContent(i); //h->GetBinContent(i)  获取第i个bin的高度 
    dtd->Fill(hdt->GetBinLowEdge(i+1), df);  //获取第i个bin的x值
}
dtd->Sumw2(0);
dtd->Draw();
dtd->Fit("gaus", "", "", -14, -10);
c->Draw();

 FCN=34.2378 FROM MIGRAD    STATUS=CONVERGED      99 CALLS         100 TOTAL
                     EDM=1.67284e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     8.35213e+01   6.21648e+00   1.55095e-02  -2.74276e-05
   2  Mean        -1.17322e+01   5.05429e-02   1.31909e-04   5.23641e-04
   3  Sigma        6.17865e-01   3.35304e-02   6.16959e-05   1.71719e-02


# $dt_l$ = -11.7322

In [6]:
TF1 *f1 = new TF1("f1", "[0] * TMath::Exp(-0.5 * ((x - [1])/[2])^2)", 40, 43);
//gaus:f(x) = p0*exp(-0.5*((x-p1)/p2)^2)

In [7]:
f1->SetParameter(0, -320);
f1->SetParameter(1, 41.5);
f1->SetParameter(2, 0.5);
dtd->Fit("f1", "R");
dtd->Draw();
c->Draw();

 FCN=28.174 FROM MIGRAD    STATUS=CONVERGED      87 CALLS          88 TOTAL
                     EDM=1.36723e-08    STRATEGY= 1  ERROR MATRIX UNCERTAINTY   2.3 per cent
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0          -8.66634e+01   6.81985e+00   5.85659e-03  -7.26040e-06
   2  p1           4.16323e+01   4.01501e-02   1.81863e-05  -4.59294e-03
   3  p2           5.53358e-01   2.70118e-02  -1.56211e-05  -2.03839e-03


# $dt_r$ = 41.6323

- $dt_l = -11.7322,$  $dt_r$ = 41.6323
- $de_c = (dt_l + dt_r)/2 = (-11.7322 + 41.6323)/2 = 14.95005$
- $C_1 = -14.95005$
- $dt_{width} = dt_r - dt_l = 53.3645$
- $C_2 = 2L/dt_{width} = 200/53.3645 = 3.74780987$
- $a = C_2 = 3.74780987$
- $b = C_1C_2 = -56.029945$

# $$x=(t_d-t_u - 14.95005)*3.74780987$$

In [8]:
TH1D *htx=new TH1D("htx", "htx", 500, -120, 120);

In [9]:
for(Long64_t jentry=0; jentry<nentries; jentry++) 
{//对每个事件进行遍历
    tree->GetEntry(jentry);
    Double_t tx = 3.74780987 * (td - tu - 14.95005);
    htx->Fill(tx);
}
htx->Draw();
c->Draw();

In [10]:
TH2D *hdx=new TH2D("hdx", "htx-hx:hx", 100, -20, 20, 500, -120, 120);

In [11]:
for(Long64_t jentry=0; jentry<nentries; jentry++) 
{//对每个事件进行遍历
    tree->GetEntry(jentry);
    Double_t tx = 3.74780987 * (td - tu - 14.95005);
    hdx->Fill(tx-x,x);//difference
}
hdx->Draw("colz");
c->Draw();

In [12]:
TH1D *hdx_X=hdx->ProjectionX("projx of hdx");
hdx_X->Draw();
hdx_X->Fit("gaus");//中心值 mean dx=0.2, 说明确定td-tu的边界时存在微小偏差。
c->Draw();

 FCN=52.2385 FROM MIGRAD    STATUS=CONVERGED      49 CALLS          50 TOTAL
                     EDM=1.98011e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     7.08741e+03   2.73606e+01   7.98697e-02  -3.23821e-06
   2  Mean        -1.88279e-01   7.11954e-03   2.53643e-05  -8.76454e-02
   3  Sigma        2.25041e+00   4.98148e-03   2.15270e-06  -1.11379e-01


   ###       1.2 利用电荷信息确定位置$q_x$

### 探测器两端能量(x 处能量沉积$Q_0$):
- $Q_u=Q_0 exp[-(L-x)/\lambda]$
- $Q_d=Q_0 exp[-(L+x)/\lambda]$
- $x=\frac{\lambda}{2}ln\frac{q_u}{q_d}$

#  $$x=C * ln\frac{q_u}{q_d}$$

结论：位置与$ln\frac{q_u}{q_d}$成正比

In [13]:
tree->Draw("log(qu/qd):x", "", "colz");
c->Draw();

结论：上图印证了位置与$ln\frac{q_u}{q_d}$成正比

In [14]:
tree->Draw("log(qu/qd)>>hlogq(80, -0.8, 0.8)");
c->Draw();

# 微分法确定时间差$ln\frac{q_u}{q_d}$的边界

In [15]:
TH1D *dlogqd = new TH1D("dlogqd", "dlogq/dx", 40, -0.78, 0.82);

In [16]:
for(int i=1; i<hlogq->GetNbinsX(); i++) 
{   //h->GetNbinsX()  获取 bin 总数   bin数从1开始
    Double_t dlogq = hlogq->GetBinContent(i+1) - hlogq->GetBinContent(i); //h->GetBinContent(i)  获取第i个bin的高度 
    dlogqd->Fill(hlogq->GetBinLowEdge(i+1), dlogq);  //获取第i个bin的x值
}
dlogqd->Sumw2(0);
dlogqd->Draw();
dlogqd->Fit("gaus", "", "", -0.7, -0.35);
c->Draw();

 FCN=481.128 FROM MIGRAD    STATUS=CONVERGED      68 CALLS          69 TOTAL
                     EDM=8.8512e-09    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     4.07047e+02   1.19763e+01   1.10172e-01  -6.36567e-06
   2  Mean        -5.40136e-01   1.92923e-03   2.02877e-05   2.63124e-02
   3  Sigma        6.51280e-02   1.31957e-03   5.88178e-05   9.39291e-03


# $dlogq_l$ = -0.540136

In [17]:
TF1 *f2 = new TF1("f1", "[0] * TMath::Exp(-0.5 * ((x - [1])/[2])^2)", 0.4, 0.7);

In [18]:
f2->SetParameter(0, -400);
f2->SetParameter(1, 0.5);
f2->SetParameter(2, 0.5);
dlogqd->Fit("f1", "R");
dlogqd->Draw();
c->Draw();

 FCN=36.6466 FROM MIGRAD    STATUS=CONVERGED     387 CALLS         388 TOTAL
                     EDM=4.7933e-09    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0          -4.88738e+02   1.43270e+01   3.46918e-02  -5.05444e-06
   2  p1           5.32814e-01   1.68911e-03   4.62009e-06   2.79143e-02
   3  p2           5.99512e-02   1.26248e-03   2.84825e-06  -5.06610e-03


# $dlogq_r$ = 0.532814

- $dlogq = \frac{dlogq_r-dlogq_l}{2} = \frac{0.532814+0.540136}{2} = 0.536475$

# $$C = \frac{L}{dlogq} = \frac{100}{0.536475} = 186.401976$$

In [19]:
TH1D *hqx=new TH1D("hqx", "hqx", 500, -120, 120);

In [20]:
for(Long64_t jentry=0; jentry<nentries; jentry++) 
{//对每个事件进行遍历
    tree->GetEntry(jentry);
    Double_t qx = 186.401976 * log(qu/qd);
    hqx->Fill(qx);
}
hqx->Draw();
c->Draw();

In [21]:
TH2D *hdqx=new TH2D("hdqx", "hqx-hx:hx", 100, -40, 40, 500, -120, 120);

In [22]:
for(Long64_t jentry=0; jentry<nentries; jentry++) 
{//对每个事件进行遍历
    tree->GetEntry(jentry);
    Double_t qx = 186.401976 * log(qu/qd);
    hdqx->Fill(qx-x,x);//difference
}
hdqx->Draw("colz");
c->Draw();

In [23]:
TH1D *hdqx_X=hdqx->ProjectionX("projx of hdx");
hdqx_X->Draw();
hdqx_X->Fit("gaus");//中心值 mean dx=-0.066, 说明确定td-tu的边界时存在微小偏差。
c->Draw();

 FCN=98.6308 FROM MIGRAD    STATUS=CONVERGED      60 CALLS          61 TOTAL
                     EDM=1.45659e-14    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     2.82033e+03   1.09541e+01   4.34985e-02   1.06691e-08
   2  Mean        -6.62842e-02   3.58795e-02   1.74826e-04  -1.86571e-06
   3  Sigma        1.13047e+01   2.55940e-02   2.99818e-06  -7.75554e-05


# $t_x,$ $q_x$的结果比较

- 根据$t_x,$ $q_x$残差图的结果，$q_x$的结果比$t_x$的准确性要高一个量级，但是$t_x$的精确性要比$q_x$高一个量级，则如果是大统计量确定粒子的注入位置，应该选择$q_x$,其结果更加可靠，对于小统计量$q_x$得出的结果涨落比较严重，应该选择$t_x$来确定粒子的注入位置。

# 2.计算中子能量e

### 2.1TOF绝对刻度
实际探测器两端时间信号：
- 实际时间信号还具有一个额外附加的常数项，$T_{Off}$：来源包括PMT度越时间，和电缆，电子学的传输时间。
 - 这个常数项一般是无法直接测定的，需要由实验数据来计算得到。
- $t_u=TOF+(L-x)/v_{sc}+T_{uoff}$
- $t_d=TOF+(L+x)/v_{sc}+T_{doff}$
- $TOF=(t_u+t_d)/2 -L/v_{sc}-(T_{uoff}+T_{doff})/2$
- 将上式常数项设为C，$TOF=(t_u+t_d)/2 +C$

#### 结论：$TOF$与$(t_u+t_d)/2$只相差一个常数C

$\gamma$ 飞行时间：$TOF_{\gamma}(ns)=3.333\cdot d(m)$

#### 光子打到探测器中心时，理论准确飞行时间$TOF_0(ns)=3.333\cdot d_0(m)$,其中 $d_0$为靶点到探测器中心距离。 与$(t_u+t_d)/2$比较可以确定常数C
- 已知中心平均飞行距离502.5cm,则 $TOF_0 = 3.333 × 5.025 = 16.748325 ns$

In [24]:
c->SetLogy();
tree->Draw("(tu+td)/2>>tc(1600, 0, 160)");
c->Draw()

In [25]:
TH2D *hgtofc_tx = new TH2D("hgtofc_tx", "hgtofc_tx", 100, -120, 120, 100, 39, 45);

In [26]:
c->SetLogy(0);
for(Long64_t jentry=0; jentry<nentries; jentry++) 
{//对每个事件进行遍历
    tree->GetEntry(jentry);
    Double_t tx = 3.74780987 * (td - tu - 14.95005);
    if(((tu+td)/2)>40 && ((tu+td)/2)<45) 
        hgtofc_tx->Fill(tx, ((tu+td)/2));
}
hgtofc_tx->Draw("colz");
c->Draw();

In [27]:
TH1D *hgctof=new TH1D("hgctof", "hgctof", 100, 38, 45);

In [28]:
for(Long64_t jentry=0; jentry<nentries; jentry++) 
{//对每个事件进行遍历
    tree->GetEntry(jentry);
    Double_t tx = 3.74780987 * (td - tu - 14.95005);
    if(((tu+td)/2)>40 && ((tu+td)/2)<45 && abs(tx)<5) 
        hgctof->Fill(ctof);//gamma hits the center of the det.
}
hgctof->Draw();
hgctof->Fit("gaus");
c->Draw();

 FCN=26.3546 FROM MIGRAD    STATUS=CONVERGED      60 CALLS          61 TOTAL
                     EDM=1.33765e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     1.38871e+02   4.52573e+00   9.17387e-03  -3.64970e-06
   2  Mean         4.12741e+01   7.91373e-03   2.02048e-05  -1.99959e-02
   3  Sigma        3.01508e-01   6.14876e-03   1.36766e-05   5.02898e-03


- $TOF(0)=41.2745$ ns
- $C = TOF_0 - TOF(0) = 16.748325 - 41.2745 = -24.526175$ ns

### 2.1TOF绝对刻度

中子飞行时间：$TOF_n(ns)=\frac{72 \cdot d(m)}{\sqrt{E_n(MeV)}}$

则中子能量为：

# $$E_n(MeV) =  {（\frac{72 \cdot d(m)}{TOF_n(ns)}）}^2$$

In [29]:
TH1D *htofc=new TH1D("htofc", "htofc", 90, 0, 30);
TH1D *hne=new TH1D("hne", "hne", 600, -50, 250);

In [30]:
c->SetLogy();
for(Long64_t jentry=0; jentry<nentries; jentry++) 
{//对每个事件进行遍历
    tree->GetEntry(jentry);
    Double_t tofc=((tu + td) / 2) - 24.526175;
    Double_t tx = 3.74780987 * (td - tu - 14.95005);
    Double_t d = TMath::Sqrt(502.5 * 502.5 + tx * tx);
    Double_t tofcn = tofc/d *100.0;//normalized to 100cm
    htofc->Fill(tofcn);
    if(((tu+td)/2) > 45)
    {
        Double_t ne = pow((72 / tofcn), 2);
        hne->Fill(ne);
    }
}
htofc->Draw();
c->Draw();

In [31]:
c->SetLogy();
hne->Fit("gaus");
hne->Draw();
c->Draw();

 FCN=264.742 FROM MIGRAD    STATUS=CONVERGED      50 CALLS          51 TOTAL
                     EDM=8.73857e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     7.47247e+02   3.47121e+00   2.25223e-02   4.32211e-04
   2  Mean         9.16869e+01   7.06880e-02   5.62459e-04  -6.24330e-03
   3  Sigma        1.86358e+01   5.03345e-02   5.82424e-06   6.81834e-01


# 则中子的能量为：

# $$Energy_n = 91.69(1863) MeV$$
